## Load shared adapter helpers

The next cell pulls in the same table builders the loader and transformer use so we talk to the exact same Iceberg catalogues.

In [ ]:
import pandas as pd

from adapters.axiell.steps.loader import AXIELL_NAMESPACE
from adapters.axiell.table_config import get_iceberg_table
from adapters.axiell.window_status import build_window_store
from adapters.utils.iceberg import IcebergTableClient


RECORDS_TABLE = get_iceberg_table(use_rest_api_table=False)

records_client = IcebergTableClient(
    RECORDS_TABLE,
    default_namespace=AXIELL_NAMESPACE,
)

window_store = build_window_store(use_rest_api_table=False)

print("Records table info:")
print(RECORDS_TABLE)

print("\nWindow status table info:")
print(window_store.table)

DISPLAY_LIMIT = 20

## Preview harvested records

Use the same `IcebergTableClient` API the lambdas employ to view the latest rows from the records table.

In [ ]:
def preview_records(limit: int = DISPLAY_LIMIT, include_deleted: bool = False) -> pd.DataFrame:

    """Return a small slice of the records table as a DataFrame."""

    table = records_client.get_all_records(include_deleted=include_deleted)
    if table is None or table.num_rows == 0:
        return pd.DataFrame(
            columns=["namespace", "id", "content_preview", "changeset", "last_modified"]
        )

    limited = table.slice(0, limit) if limit is not None else table
    df = limited.to_pandas()

    if "content" in df.columns:
        df["content_preview"] = df["content"].astype(str).str.slice(0, 200)

    return df[["namespace", "id", "content_preview", "changeset", "last_modified"]]





preview_records(limit=5)

## Inspect a specific changeset

Set `EXAMPLE_CHANGESET` to any ID emitted by the loader to pull back just those rows.

In [ ]:
def load_changeset(changeset_id: str, limit: int | None = None) -> pd.DataFrame:

    """Fetch rows for a particular changeset identifier."""
    table = records_client.get_records_by_changeset(changeset_id)

    if table is None or table.num_rows == 0:
        return pd.DataFrame(
            columns=["namespace", "id", "content", "changeset", "last_modified"]
        )

    limited = table.slice(0, limit) if limit is not None else table

    df = limited.to_pandas()

    if "content" in df.columns:
        df["content_preview"] = df["content"].astype(str).str.slice(0, 200)

    return df[["namespace", "id", "content_preview", "changeset", "last_modified"]]


EXAMPLE_CHANGESET = "286f7e48-c3cd-11f0-bdae-2ec5165eca95"  # Replace with "abcd-..." to preview one set of writes

df = load_changeset(EXAMPLE_CHANGESET)
df

## Inspect window status rows

These helpers wrap `IcebergWindowStore` so you can see which windows have succeeded or failed locally.

In [ ]:
def window_status_frame(limit: int = 25, state: str | None = None) -> pd.DataFrame:

    """Return recent window status rows from the IcebergWindowStore."""
    rows = list(window_store.load_status_map().values())

    if not rows:
        return pd.DataFrame(
            columns=[
                "window_key",
                "window_start",
                "window_end",
                "state",
                "attempts",
                "last_error",
                "record_ids",
                "updated_at",
            ]
        )

    df = pd.DataFrame(rows)

    for column in ("window_start", "window_end", "updated_at"):
        if column in df.columns:
            df[column] = pd.to_datetime(df[column])

    df = df.sort_values("window_start", ascending=False)

    if state is not None:
        df = df[df["state"] == state]

    if limit is not None:
        df = df.head(limit)

    return df.reset_index(drop=True)

window_status_frame(limit=10)